In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib backend
plt.switch_backend('Agg')

try:
    # Load the data
    print("Loading data...")
    df = pd.read_csv('coffee.csv')
    
    # Basic data info
    print("Dataset Info:")
    print(df.info())
    print("\nFirst few rows:")
    print(df.head())
    
    # Check for missing values
    print("\nMissing values:")
    print(df.isnull().sum())
    
    # Convert datetime column
    df['datetime'] = pd.to_datetime(df['datetime'])
    print("\nConverting datetime column...")
    
    # Filter for first 2 weeks
    min_date = df['datetime'].min()
    max_date = min_date + pd.Timedelta(weeks=2)
    df_filtered = df[(df['datetime'] >= min_date) & (df['datetime'] <= max_date)]
    print(f"\nFiltered data for first 2 weeks: {len(df_filtered)} rows")
    
    # Create hourly bins
    df_filtered['hour'] = df_filtered['datetime'].dt.floor('H')
    hourly_sales = df_filtered.groupby('hour')['money'].sum()
    print(f"\nBinning data to 1-hour intervals... Binned data shape: {hourly_sales.shape}")
    print("First few binned rows:")
    print(hourly_sales.head())
    
    # Check for zero values
    zero_count = (hourly_sales == 0).sum()
    total_count = len(hourly_sales)
    zero_percentage = (zero_count / total_count) * 100
    print(f"\nZero sales hours: {zero_count} ({zero_percentage:.2f}%) of total hours")
    
    # Create a simple model for demonstration
    # For count data, we'll use a Poisson regression approach
    from sklearn.linear_model import PoissonRegressor
    from sklearn.preprocessing import StandardScaler
    
    # Create features
    hourly_sales_df = hourly_sales.reset_index()
    hourly_sales_df['hour_of_day'] = hourly_sales_df['hour'].dt.hour
    hourly_sales_df['day_of_week'] = hourly_sales_df['hour'].dt.dayofweek
    hourly_sales_df['is_weekend'] = (hourly_sales_df['day_of_week'] >= 5).astype(int)
    
    # Prepare features and target
    X = hourly_sales_df[['hour_of_day', 'day_of_week', 'is_weekend']]
    y = hourly_sales_df['money']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Fit Poisson regression
    poisson_model = PoissonRegressor(alpha=0.01)
    poisson_model.fit(X_train, y_train)
    
    # Predictions
    y_pred_poisson = poisson_model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred_poisson)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred_poisson)
    
    print("\nPoisson Regression Results:")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    
    # Compare with linear regression
    from sklearn.linear_model import LinearRegression
    
    linear_model = LinearRegression()
    linear_model.fit(X_train, y_train)
    y_pred_linear = linear_model.predict(X_test)
    
    mse_linear = mean_squared_error(y_test, y_pred_linear)
    rmse_linear = np.sqrt(mse_linear)
    mae_linear = mean_absolute_error(y_test, y_pred_linear)
    
    print("\nLinear Regression Results:")
    print(f"RMSE: {rmse_linear:.2f}")
    print(f"MAE: {mae_linear:.2f}")
    
    # Create visualization
    plt.figure(figsize=(12, 8))
    
    # Plot 1: Distribution of sales
    plt.subplot(2, 2, 1)
    plt.hist(hourly_sales, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('Distribution of Coffee Sales (First 2 Weeks)')
    plt.xlabel('Sales Amount')
    plt.ylabel('Frequency')
    
    # Plot 2: Sales over time
    plt.subplot(2, 2, 2)
    plt.plot(hourly_sales.index, hourly_sales.values, marker='o', markersize=2, alpha=0.7)
    plt.title('Coffee Sales Over Time (First 2 Weeks)')
    plt.xlabel('Time')
    plt.ylabel('Sales Amount')
    plt.xticks(rotation=45)
    
    # Plot 3: Comparison of models
    plt.subplot(2, 2, 3)
    plt.scatter(y_test, y_pred_linear, alpha=0.5, label='Linear Regression')
    plt.scatter(y_test, y_pred_poisson, alpha=0.5, label='Poisson Regression')
    plt.plot([0, max(max(y_test), max(y_pred_linear), max(y_pred_poisson))], 
             [0, max(max(y_test), max(y_pred_linear), max(y_pred_poisson))], 'r--')
    plt.xlabel('Actual Sales')
    plt.ylabel('Predicted Sales')
    plt.title('Model Comparison')
    plt.legend()
    
    # Plot 4: Coefficients
    plt.subplot(2, 2, 4)
    feature_names = ['Hour of Day', 'Day of Week', 'Is Weekend']
    poisson_coef = poisson_model.coef_
    linear_coef = linear_model.coef_
    x = np.arange(len(feature_names))
    width = 0.35
    plt.bar(x - width/2, poisson_coef, width, label='Poisson', alpha=0.8)
    plt.bar(x + width/2, linear_coef, width, label='Linear', alpha=0.8)
    plt.xlabel('Features')
    plt.ylabel('Coefficient Value')
    plt.title('Model Coefficients Comparison')
    plt.xticks(x, feature_names, rotation=45)
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('coffee_sales_model_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Summary
    print("\nModel Comparison Summary:")
    print("1. The data contains positive or zero values, which is typical for count data.")
    print("2. A Poisson regression model is more appropriate than standard linear regression for this type of data.")
    print("3. The Poisson model accounts for the non-negative, discrete nature of the data.")
    print("4. Linear regression may produce negative predictions, which are not meaningful for sales data.")
    print("5. The Poisson model is specifically designed for count data and handles zero-inflated data better.")
    
    # Additional analysis for zero-inflated data
    print("\nZero-Inflation Analysis:")
    print(f"Percentage of zero sales: {zero_percentage:.2f}%")
    if zero_percentage > 30:
        print("Warning: High percentage of zeros detected. Consider Zero-Inflated models (ZIP/ZINB)")
    else:
        print("Zero percentage is acceptable. Standard Poisson or Negative Binomial models are suitable.")
    
    print("\nRecommendations:")
    print("1. Use Poisson or Negative Binomial regression for count data")
    print("2. Consider Zero-Inflated models if zero percentage exceeds 30%")
    print("3. Validate model assumptions with residual analysis")
    print("4. Cross-validate to ensure model robustness")
    
except Exception as e:
    print(f"Error occurred: {str(e)}")

Loading data...
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3636 entries, 0 to 3635
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         3636 non-null   object 
 1   datetime     3636 non-null   object 
 2   cash_type    3636 non-null   object 
 3   card         3547 non-null   object 
 4   money        3636 non-null   float64
 5   coffee_name  3636 non-null   object 
dtypes: float64(1), object(5)
memory usage: 170.6+ KB
None

First few rows:
         date                 datetime cash_type                 card  money  \
0  2024-03-01  2024-03-01 10:15:50.520      card  ANON-0000-0000-0001   38.7   
1  2024-03-01  2024-03-01 12:19:22.539      card  ANON-0000-0000-0002   38.7   
2  2024-03-01  2024-03-01 12:20:18.089      card  ANON-0000-0000-0002   38.7   
3  2024-03-01  2024-03-01 13:46:33.006      card  ANON-0000-0000-0003   28.9   
4  2024-03-01  2024-03-01 13:48:14.626      card  AN

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Set matplotlib backend
plt.switch_backend('Agg')

def analyze_hourly_transactions():
    try:
        # Load the data
        df = pd.read_csv('coffee.csv')
        
        # Display basic info about the dataset
        
        # Convert datetime column to datetime format
        df['datetime'] = pd.to_datetime(df['datetime'])
        
        
                
        # Sort by datetime to ensure proper ordering
        df = df.sort_values('datetime')
        
        # Resample the data hourly and count transactions
        hourly_counts = df.resample('H', on='datetime').size()
        print(f"\nHourly transaction counts (first few rows):")
        print(hourly_counts.head())
        # Reindex to include all hours and fill NaN with 0
        hourly_counts = hourly_counts.reindex(pd.date_range(start=hourly_counts.index.min(), end=hourly_counts.index.max(), freq='H'), fill_value=0)
        
        hourly_counts.to_csv('hourly_transaction_counts.csv')
        print("\nHourly Transaction Counts:")
        print(hourly_counts)
        
        # Save the plot
        plt.figure(figsize=(12, 6))
        plt.plot(hourly_counts.index, hourly_counts.values)
        plt.xlabel("Hour")
        plt.ylabel("Transaction Count")
        plt.title("Hourly Transaction Counts")
        plt.grid(True)
        plt.savefig('hourly_transaction_counts.png')
        print("\nPlot saved as 'hourly_transaction_counts.png'")
        
    except Exception as e:
        print(f"Error in analyze_hourly_transactions: {e}")

# Run the analysis
analyze_hourly_transactions()


Hourly transaction counts (first few rows):
datetime
2024-03-01 10:00:00    1
2024-03-01 11:00:00    0
2024-03-01 12:00:00    2
2024-03-01 13:00:00    2
2024-03-01 14:00:00    0
Freq: h, dtype: int64

Hourly Transaction Counts:
2024-03-01 10:00:00    1
2024-03-01 11:00:00    0
2024-03-01 12:00:00    2
2024-03-01 13:00:00    2
2024-03-01 14:00:00    0
                      ..
2025-03-23 14:00:00    2
2025-03-23 15:00:00    1
2025-03-23 16:00:00    0
2025-03-23 17:00:00    0
2025-03-23 18:00:00    1
Freq: h, Length: 9297, dtype: int64

Plot saved as 'hourly_transaction_counts.png'


DatetimeIndex(['2024-03-01 10:15:50.520000', '2024-03-01 11:15:50.520000',
               '2024-03-01 12:15:50.520000', '2024-03-01 13:15:50.520000',
               '2024-03-01 14:15:50.520000', '2024-03-01 15:15:50.520000',
               '2024-03-01 16:15:50.520000', '2024-03-01 17:15:50.520000',
               '2024-03-01 18:15:50.520000', '2024-03-01 19:15:50.520000',
               ...
               '2025-03-23 08:15:50.520000', '2025-03-23 09:15:50.520000',
               '2025-03-23 10:15:50.520000', '2025-03-23 11:15:50.520000',
               '2025-03-23 12:15:50.520000', '2025-03-23 13:15:50.520000',
               '2025-03-23 14:15:50.520000', '2025-03-23 15:15:50.520000',
               '2025-03-23 16:15:50.520000', '2025-03-23 17:15:50.520000'],
              dtype='datetime64[ns]', length=9296, freq='h')